In [3]:
import sys
import pandas as pd
from datetime import datetime
from python_calamine import CalamineWorkbook
from dateutil.parser import isoparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# === File paths ===
input_file = r"C:\Users\Joshua Almari\Documents\Almari Files\Files\QA Report\Audit_Data_New500.xlsx"
gpp_file   = r"C:\Users\Joshua Almari\Documents\Almari Files\Files\QA Report\GPP.csv"
output_file = r"C:\Users\Joshua Almari\Documents\Almari Files\Files\QA Report\Audit_Data_Processed.xlsx"
sheet_name = 'Sheet1'
column_name = 'DateTimeStamp'

# === Output format toggle ===
OUTPUT_FORMAT = "csv"   # options: "csv" or "excel"

# === Short date format ===
if sys.platform.startswith('win'):
    DATE_FORMAT = "%#m/%#d/%Y"
else:
    DATE_FORMAT = "%-m/%-d/%Y"

# === Load Excel ===
wb = CalamineWorkbook.from_path(input_file)
sh = wb.get_sheet_by_name(sheet_name)
rows = sh.to_python()
df = pd.DataFrame(rows[1:], columns=rows[0])

correction_logs = []

# === Conversion helpers ===
def try_parse_formats(val_str, formats):
    for fmt in formats:
        try:
            return datetime.strptime(val_str.split(" ")[0], fmt)
        except:
            continue
    return None

def convert_to_date_obj(value):
    if pd.isna(value):
        return None
    if isinstance(value, float) and value.is_integer():
        value = int(value)
    if isinstance(value, str) and value.strip().replace(".0", "").isdigit():
        value = int(float(value.strip()))
    if isinstance(value, int):
        if value >= 10**12:  # ms timestamp
            return datetime.fromtimestamp(value / 1000).date()
        elif value >= 10**9:  # s timestamp
            return datetime.fromtimestamp(value).date()
    if isinstance(value, datetime):
        return value.date()
    val_str = str(value).strip()
    try:
        return isoparse(val_str).date()
    except:
        pass
    for fmt in ["%Y-%m-%d", "%m/%d/%Y", "%d/%m/%Y"]:
        try:
            return datetime.strptime(val_str.split(" ")[0], fmt).date()
        except:
            continue
    return None

def get_actual_data_range(df, sample_size=10):
    samples = list(df[column_name].head(sample_size)) + list(df[column_name].tail(sample_size))
    dates = [d for val in samples if (d := convert_to_date_obj(val))]
    if dates:
        return min(dates), max(dates)
    return None, None

actual_start, actual_end = get_actual_data_range(df)
print(f"✅ Actual data range: {actual_start} → {actual_end}")

def convert_to_date(value, actual_start, actual_end):
    def force_in_range(dt, orig_value):
        if actual_start <= dt.date() <= actual_end:
            return dt
        try:
            alt_dt = dt.replace(day=dt.month, month=dt.day)
            if actual_start <= alt_dt.date() <= actual_end:
                correction_logs.append(f"🔄 Reinterpreted {orig_value} → {alt_dt.strftime(DATE_FORMAT)} (swap)")
                return alt_dt
        except:
            pass
        if dt.date() < actual_start:
            adjusted = dt + (actual_start - dt.date())
            correction_logs.append(f"🔄 Forced {orig_value} → {adjusted.strftime(DATE_FORMAT)} (to start)")
            return adjusted
        elif dt.date() > actual_end:
            adjusted = dt - (dt.date() - actual_end)
            correction_logs.append(f"🔄 Forced {orig_value} → {adjusted.strftime(DATE_FORMAT)} (to end)")
            return adjusted
        return dt

    try:
        if pd.isna(value):
            return None
        val_str = str(value).strip()
        try:
            dt = isoparse(val_str)
            return force_in_range(dt, val_str).date()
        except:
            pass
        dt = try_parse_formats(val_str, ["%Y-%m-%d", "%m/%d/%Y", "%d/%m/%Y"])
        if dt:
            return force_in_range(dt, val_str).date()
        try:
            if isinstance(value, (int, float)):
                if value >= 10**12:
                    dt = datetime.fromtimestamp(int(value) / 1000)
                elif value >= 10**9:
                    dt = datetime.fromtimestamp(int(value))
                else:
                    return None
                return force_in_range(dt, val_str).date()
        except:
            pass
        correction_logs.append(f"⚠️ Could not parse {val_str}, left as-is")
        return None
    except Exception as e:
        correction_logs.append(f"❌ Error parsing {value!r}: {e}")
        return None

# === Parallel processing ===
converted_dates = [None] * len(df)
with ThreadPoolExecutor() as executor:
    futures = {
        executor.submit(convert_to_date, val, actual_start, actual_end): idx
        for idx, val in enumerate(df[column_name])
    }
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Dates"):
        idx = futures[future]
        try:
            converted_dates[idx] = future.result()
        except Exception as e:
            converted_dates[idx] = None
            correction_logs.append(f"❌ Error parsing index {idx}: {e}")

df[column_name] = converted_dates

# Drop Column1 if exists
if "Column1" in df.columns:
    df = df.drop(columns=["Column1"])

# Ensure datetime column
df[column_name] = pd.to_datetime(df[column_name], errors='coerce')

# --- Filters (use only one at a time) ---
TARGET_DATE = '2025-09-24'    # for single dates '2025-09-08'
TARGET_RANGE = None   # for multiple dates ('2025-09-05', '2025-09-07')

if TARGET_DATE:
    target_date = pd.to_datetime(TARGET_DATE)
    df_filtered = df[df[column_name] == target_date].copy()
elif TARGET_RANGE:
    start_date = pd.to_datetime(TARGET_RANGE[0])
    end_date   = pd.to_datetime(TARGET_RANGE[1])
    df_filtered = df[(df[column_name] >= start_date) & (df[column_name] <= end_date)].copy()
else:
    df_filtered = df.copy()

# === Merge with GPP file ===
if gpp_file.lower().endswith(".csv"):
    gpp_df = pd.read_csv(gpp_file, usecols=["Global ID", "Remote Partner Location", "Is Agent Team Lead", "Buffer Resource", "Start Date"])
else:
    gpp_sheet = "GPP"
    gpp_df = pd.read_excel(gpp_file, sheet_name=gpp_sheet, usecols=["Global ID", "Remote Partner Location", "Is Agent Team Lead", "Buffer Resource", "Start Date"])

gpp_df["Global ID"] = gpp_df["Global ID"].astype(str).str.strip()
df_filtered.loc[:, "GlobalID"] = df_filtered["GlobalID"].astype(str).str.strip()

merged = df_filtered.merge(gpp_df, left_on="GlobalID", right_on="Global ID", how="left")
merged = merged.drop(columns=["Global ID"])
merged["Remote Partner Location"] = merged["Remote Partner Location"].fillna("")
merged = merged[~merged["Remote Partner Location"].isin(["Mohali", "Jaipur"])]

# --- Save file (CSV or Excel) ---
if OUTPUT_FORMAT.lower() == "csv":
    merged.loc[:, column_name] = merged[column_name].dt.strftime(DATE_FORMAT)
    csv_file = output_file.replace(".xlsx", ".csv")
    merged.to_csv(csv_file, index=False)
    print(f"✅ Done. Filtered + Merged CSV saved to:\n{csv_file}")

elif OUTPUT_FORMAT.lower() == "excel":
    with pd.ExcelWriter(output_file.replace(".csv", ".xlsx"), engine="xlsxwriter", datetime_format=DATE_FORMAT) as writer:
        merged.to_excel(writer, index=False, sheet_name=sheet_name)
        ws = writer.sheets[sheet_name]
        col_idx = merged.columns.get_loc(column_name)
        date_fmt = writer.book.add_format({'num_format': 'm/d/yyyy'})
        ws.set_column(col_idx, col_idx, 15, date_fmt)
    print(f"✅ Done. Filtered + Merged Excel file saved to:\n{output_file.replace('.csv', '.xlsx')}")

print("\n=== Date Correction Summary ===")
for log in correction_logs:
    print(log)

✅ Actual data range: 2025-07-16 → 2025-09-25


Processing Dates: 100%|████████████████████████████████████████████████████| 248972/248972 [00:02<00:00, 122430.92it/s]


✅ Done. Filtered + Merged CSV saved to:
G:\Shared drives\RA Shared Folder\RA Tasks\Almari Files\Files\QA Report\Audit_Data_Processed.csv

=== Date Correction Summary ===
🔄 Reinterpreted 2025-01-08T21:01:00.000Z → 8/1/2025 (swap)
🔄 Reinterpreted 2025-01-08T21:13:00.000Z → 8/1/2025 (swap)
🔄 Reinterpreted 2025-04-08T15:22:00.000Z → 8/4/2025 (swap)
🔄 Reinterpreted 2025-04-08T15:24:00.000Z → 8/4/2025 (swap)
🔄 Reinterpreted 2025-04-08T15:31:00.000Z → 8/4/2025 (swap)
🔄 Reinterpreted 2025-04-08T15:39:00.000Z → 8/4/2025 (swap)
🔄 Reinterpreted 2025-04-08T21:27:00.000Z → 8/4/2025 (swap)
🔄 Reinterpreted 2025-05-08T15:48:00.000Z → 8/5/2025 (swap)
🔄 Reinterpreted 2025-05-08T15:52:00.000Z → 8/5/2025 (swap)
🔄 Reinterpreted 2025-05-08T16:58:00.000Z → 8/5/2025 (swap)
🔄 Reinterpreted 2025-05-08T20:18:00.000Z → 8/5/2025 (swap)
🔄 Reinterpreted 2025-05-08T20:59:00.000Z → 8/5/2025 (swap)
🔄 Reinterpreted 2025-06-08T15:07:00.000Z → 8/6/2025 (swap)
🔄 Reinterpreted 2025-06-08T15:33:00.000Z → 8/6/2025 (swap)
🔄 Re

In [4]:
merged

,GlobalID,FirstName,LastName,RemotePartner,Role,Team Lead,DomesticTeamLead1,DomesticTeamLead2,Sales Order Number,Invoice Number,...,Score,Auditor,Auditor Comment,DateTimeStamp,Role_Selection,__PowerAppsId__,Start Date,Remote Partner Location,Is Agent Team Lead,Buffer Resource
0,AH91014565,Lynette,Salonga,Knack,Insurance Verification,Jenny Guinto,Kelli Ore,Rick Adamich,30974110.0,,...,100.0,baby.gavino@adapthealth.com,,2025-09-24,Insurance Verification,b86915ef-0618-4a64-af13-f946a4131f08,06/24/2025,Clark,0,0
1,AH91014565,Lynette,Salonga,Knack,Insurance Verification,Jenny Guinto,Kelli Ore,Rick Adamich,30969233.0,,...,100.0,baby.gavino@adapthealth.com,,2025-09-24,Insurance Verification,0ae41bd3-9acd-46a6-aa9c-23b9e59cf86a,06/24/2025,Clark,0,0
2,AH91014565,Lynette,Salonga,Knack,Insurance Verification,Jenny Guinto,Kelli Ore,Rick Adamich,30967246.0,,...,100.0,baby.gavino@adapthealth.com,,2025-09-24,Insurance Verification,85f9c3c5-1867-4705-9b41-010b560b69f8,06/24/2025,Clark,0,0
3,AH91014565,Lynette,Salonga,Knack,Insurance Verification,Jenny Guinto,Kelli Ore,Rick Adamich,,,...,100.0,baby.gavino@adapthealth.com,,2025-09-24,Insurance Verification,276f3a1d-655d-42b1-a8e0-85a4e15372dd,06/24/2025,Clark,0,0
4,AH91014565,Lynette,Salonga,Knack,Insurance Verification,Jenny Guinto,Kelli Ore,Rick Adamich,30967512.0,,...,100.0,baby.gavino@adapthealth.com,,2025-09-24,Insurance Verification,28425ca8-c649-45d1-940d-47a9a687fd57,06/24/2025,Clark,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,AH91014986,Kevin,Manabat,Knack,Documentation Recerts,Norman Cuaresma,Samantha Parton,Kelly Bratz,10322998.0,,...,100.0,eijaybon.magtoto@adapthealth.com,,2025-09-24,Documentation Recertification,3a1423ba-e693-4060-898d-84d58eea306e,08/05/2025,Clark,0,0
5456,AH91014986,Kevin,Manabat,Knack,Documentation Recerts,Norman Cuaresma,Samantha Parton,Kelly Bratz,29412195.0,,...,100.0,eijaybon.magtoto@adapthealth.com,,2025-09-24,Documentation Recertification,db640596-faa3-4019-abfa-4f7ed2eca7f8,08/05/2025,Clark,0,0
5457,AH91014986,Kevin,Manabat,Knack,Documentation Recerts,Norman Cuaresma,Samantha Parton,Kelly Bratz,29324144.0,,...,100.0,eijaybon.magtoto@adapthealth.com,,2025-09-24,Documentation Recertification,c80f7220-a649-4d5a-a74a-6679838c8e1c,08/05/2025,Clark,0,0
5458,AH91014987,Ryan,Manalili,Knack,Documentation Recerts,Norman Cuaresma,Samantha Parton,Kelly Bratz,12507284.0,,...,100.0,eijaybon.magtoto@adapthealth.com,,2025-09-24,Documentation Recertification,0e7da5eb-71c8-4cf1-94e7-e9c243afade5,08/05/2025,Clark,0,0
